In [ ]:
import pandas as pd
import os
import os.path
import re
import datetime

In [ ]:
# Parses a folder with Spirent test results and shows an extract of each result. Useful for comparing (e.g. test results from a series of tests)
folder = 'spirent tests'

In [ ]:
df_list = []
for path in os.listdir(folder):
    spirent_file = os.path.join(folder,path)
    # e.g. 21-02-09_08.31.53__RID-123__Test_name.xls
    match = re.search(r'(\d\d-\d\d-\d\d_\d\d.\d\d.\d\d)__RID-(\d\d\d)__(.*).xls', path)
    if match is None:
        continue
    try:
        df = pd.read_excel(spirent_file, sheet_name='L5-7 Client|Basic')
    except:
        print('Could not read L5-7 Client|Basic sheet from {0}'.format(path))
        continue
    df['file'] = path
    df['datetime'] = datetime.datetime.strptime(match.group(1), '%y-%m-%d_%H.%M.%S')
    df['test number'] = match.group(2)
    df['test name'] = match.group(3)
    
    mm_df = pd.read_excel(spirent_file, sheet_name='MM')
    sm_df = pd.read_excel(spirent_file, sheet_name='SM')
    scf_sut_info = pd.read_excel(spirent_file, sheet_name='CFG-SUT INFO', skiprows=2)
    
    df['sut name'] = scf_sut_info.iloc[0,0]
    df['sut ip'] = scf_sut_info.iloc[0,1]
    df['packet loss (%)'] = df['Basic Data Messages Lost'] / df['Basic Data Messages Sent'] * 100
    
    df = df.join(mm_df.iloc[:,3:])
    df = df.join(sm_df.iloc[:,3:])
    df_list.append(df)
df = pd.concat(df_list, ignore_index=True)
df = df.sort_values(by='datetime')
df['Basic Data Messages Lost (calculated)'] = df['Basic Data Messages Sent'] - df['Basic Data Messages Received']

In [ ]:
# Case where there is a mix of SUTs
# filtered_df = df[df['sut name']!='XXX']
filtered_df = df
summary_table = filtered_df[filtered_df['Interval']=='Current'].loc[:,[
    'test number', 'test name', 'datetime', 
    'Registration Requests', 'Registration Completes', 'Pdu Establishment Requests', 'Pdu Establishment Accepts', 
    'Basic Data Messages Sent', 'Basic Data Messages Received', 'Basic Data Messages Lost', 'Basic Data Messages Lost (calculated)', 'packet loss (%)',
    'Basic Data Message One Way Trip Delay (micro-sec)', 'Basic Data Minimum One Way Delay (micro-sec)', 'Basic Data Maximum One Way Delay (micro-sec)', 'sut ip']]

display(summary_table)
summary_table.to_clipboard(excel=True)

In [ ]:
import plotly.express as px
fig = px.bar(summary_table, x='test number', y='Basic Data Message One Way Trip Delay (micro-sec)')
fig.show()